In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# first solution:
training another yolov7 to detect and classify charachters in a number plate. the dataset is available in: https://github.com/mut-deep/IR-LPR. first step is downloading the dataset and converting its annotations to YOLO format. In this case we cannot use roboflow to generate the dataset cause the number of images exceeds the free plan limit. 

In [ ]:
# download the dataset from the shared gdrive and move it to our gdrive 
# !gdown --no-cookies 1ubkg7E2vGEOqS4K_quwf9Vl-i8IVpklM
# !gdown --no-cookies 1AL5Zsg2hDqcwF8ZmR0MJTbjgXIoE5W-I
# !gdown --no-cookies 1lLh_kxrHy1teUB2NguHVuOZwA5rjL5k

In [ ]:
# Install the PyDrive wrapper & import libraries.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# file_id = '1ubkg7E2vGEOqS4K_quwf9Vl-i8IVpklM'
# file_id = '1AL5Zsg2hDqcwF8ZmR0MJTbjgXIoE5W-I'
file_id = '1lLh_kxrHy1teUB2NguHVuOZwA5rjL5kx'
downloaded = drive.CreateFile({'id':file_id})
downloaded.FetchMetadata(fetch_all=True)
downloaded.GetContentFile(downloaded.metadata['title'])

In [ ]:
# extract datasets
%cd /content/gdrive/MyDrive/yolov7/ANPR-2/
!unzip plate_img-train.zip
!unzip plate_img-validation.zip
!unzip plate_img-test.zip


Streaming output truncated to the last 5000 lines.
  inflating: test/15282.jpg          
  inflating: test/15282.xml          
  inflating: test/15283.jpg          
  inflating: test/15283.xml          
  inflating: test/15292.jpg          
  inflating: test/15292.xml          
  inflating: test/15293.jpg          
  inflating: test/15293.xml          
  inflating: test/15302.jpg          
  inflating: test/15302.xml          
  inflating: test/15303.jpg          
  inflating: test/15303.xml          
  inflating: test/15312.jpg          
  inflating: test/15312.xml          
  inflating: test/15313.jpg          
  inflating: test/15313.xml          
  inflating: test/15322.jpg          
  inflating: test/15322.xml          
  inflating: test/15323.jpg          
  inflating: test/15323.xml          
  inflating: test/15332.jpg          
  inflating: test/15332.xml          
  inflating: test/15333.jpg          
  inflating: test/15333.xml          
  inflating: test/15342.jpg          

In [ ]:
# https://towardsdatascience.com/convert-pascal-voc-xml-to-yolo-for-object-detection-f969811ccba5
import xml.etree.ElementTree as ET
import glob
import os
import json
import time

def xml_to_yolo_bbox(bbox, w, h):
    # xmin, ymin, xmax, ymax
    x_center = ((bbox[2] + bbox[0]) / 2) / w
    y_center = ((bbox[3] + bbox[1]) / 2) / h
    width = (bbox[2] - bbox[0]) / w
    height = (bbox[3] - bbox[1]) / h
    return [x_center, y_center, width, height]
    
def yolo_to_xml_bbox(bbox, w, h):
    # x_center, y_center width heigth
    w_half_len = (bbox[2] * w) / 2
    h_half_len = (bbox[3] * h) / 2
    xmin = int((bbox[0] * w) - w_half_len)
    ymin = int((bbox[1] * h) - h_half_len)
    xmax = int((bbox[0] * w) + w_half_len)
    ymax = int((bbox[1] * h) + h_half_len)
    return [xmin, ymin, xmax, ymax]

In [ ]:
%cd /content/gdrive/MyDrive/yolov7/ANPR-2/
# train set
if not os.path.isdir("train/images"):
    os.mkdir("train/images")
if not os.path.isdir("train/labels"):
    os.mkdir("train/labels")
if not os.path.isdir("train/annotations"):
    os.mkdir("train/annotations")
# validation set
if not os.path.isdir("validation/images"):
    os.mkdir("validation/images")
if not os.path.isdir("validation/labels"):
    os.mkdir("validation/labels")
if not os.path.isdir("validation/annotations"):
    os.mkdir("validation/annotations")
# test set
if not os.path.isdir("test/images"):
    os.mkdir("test/images")
if not os.path.isdir("test/labels"):
    os.mkdir("test/labels")
if not os.path.isdir("test/annotations"):
    os.mkdir("test/annotations")

/content/gdrive/MyDrive/yolov7/ANPR-2


In [ ]:
!mv train/*.jpg train/images
!mv validation/*.jpg validation/images
!mv test/*.jpg test/images
!mv train/*.xml train/annotations
!mv validation/*.xml validation/annotations
!mv test/*.xml test/annotations

mv: cannot stat 'train/*.jpg': No such file or directory
mv: cannot stat 'validation/*.jpg': No such file or directory
mv: cannot stat 'test/*.jpg': No such file or directory
mv: cannot stat 'train/*.xml': No such file or directory
mv: cannot stat 'validation/*.xml': No such file or directory
mv: cannot stat 'test/*.xml': No such file or directory


In [ ]:
# process train set annotations
classes = []
input_dir = "train/annotations"
output_dir = "train/labels/"
image_dir = "train/images/"
if not os.path.isdir(output_dir):
    os.mkdir(output_dir)

In [ ]:
# import cv2 as cv
from PIL import Image
import time
files = glob.glob(os.path.join(input_dir, '*.xml'))
for fil in files:
    basename = os.path.basename(fil)
    filename = os.path.splitext(basename)[0]
    print(filename, time.strftime("%I:%M:%S"))
    if not os.path.exists(os.path.join(image_dir, f"{filename}.jpg")):
        print(f"{filename} image does not exist!")
        continue
    
    # im  = cv.imread(os.path.join(image_dir, f"{filename}.jpg"))
    # height, width, c = im.shape

    # PIL is way more faster than cv2 in opening files
    im = Image.open(os.path.join(image_dir, f"{filename}.jpg"))
    height, width = im.height, im.width
    
    result = []
    # parse the content of the xml file
    tree = ET.parse(fil)
    root = tree.getroot()


    for obj in root.findall('object'):
        label = obj.find("name").text
        # check for new classes and append to list
        if label not in classes:
            classes.append(label)
        index = classes.index(label)
        pil_bbox = [int(x.text) for x in obj.find("bndbox")]
        yolo_bbox = xml_to_yolo_bbox(pil_bbox, width, height)
        # convert data to string
        bbox_string = " ".join([str(x) for x in yolo_bbox])
        result.append(f"{index} {bbox_string}")

    if result:
        # generate a YOLO format text file for each xml file
        with open(os.path.join(output_dir, f"{filename}.txt"), "w", encoding="utf-8") as f:
            f.write("\n".join(result))

# generate the classes file as reference
with open('classes.txt', 'w', encoding='utf8') as f:
    f.write(json.dumps(classes))


In [ ]:
with open('classes.txt', 'r', encoding='utf8') as f:
  classes = json.load(f)
print(len(classes), type(classes))

31 <class 'list'>


In [ ]:
from PIL import Image
# generate labels for validation
input_dir = "validation/annotations"
output_dir = "validation/labels/"
image_dir = "validation/images/"
files = glob.glob(os.path.join(input_dir, '*.xml'))
for fil in files:
    basename = os.path.basename(fil)
    filename = os.path.splitext(basename)[0]
    print(filename, time.strftime("%I:%M:%S"))
    if not os.path.exists(os.path.join(image_dir, f"{filename}.jpg")):
        print(f"{filename} image does not exist!")
        continue
    
    im = Image.open(os.path.join(image_dir, f"{filename}.jpg"))
    height, width = im.height, im.width
    
    result = []
    # parse the content of the xml file
    tree = ET.parse(fil)
    root = tree.getroot()


    for obj in root.findall('object'):
        label = obj.find("name").text
        # skip classes which are not seen in train data
        if label not in classes:
            continue
        index = classes.index(label)
        pil_bbox = [int(x.text) for x in obj.find("bndbox")]
        yolo_bbox = xml_to_yolo_bbox(pil_bbox, width, height)
        # convert data to string
        bbox_string = " ".join([str(x) for x in yolo_bbox])
        result.append(f"{index} {bbox_string}")

    if result:
        # generate a YOLO format text file for each xml file
        with open(os.path.join(output_dir, f"{filename}.txt"), "w", encoding="utf-8") as f:
            f.write("\n".join(result))


17825 03:33:49
17795 03:33:49
17855 03:33:50
17835 03:33:52
17845 03:33:52
17865 03:33:53
17885 03:33:54
17875 03:33:55
17895 03:33:56
17905 03:33:58
17925 03:33:59
17915 03:34:00
17945 03:34:01
17935 03:34:02
17975 03:34:03
17995 03:34:03
17965 03:34:04
17955 03:34:06
17985 03:34:07
18025 03:34:07
18015 03:34:08
18045 03:34:09
18035 03:34:10
18005 03:34:11
18055 03:34:12
18065 03:34:13
18075 03:34:14
18105 03:34:15
18095 03:34:16
18085 03:34:57
18145 03:35:14
18155 03:35:15
18125 03:35:15
18135 03:35:16
18115 03:35:17
18175 03:35:18
18185 03:35:18
18195 03:35:19
18215 03:35:20
18165 03:35:20
18205 03:35:21
18275 03:35:22
18245 03:35:22
18265 03:35:23
18255 03:35:24
18235 03:35:24
18225 03:35:25
18325 03:35:26
18285 03:35:26
18315 03:35:27
18335 03:35:28
18305 03:35:28
18295 03:35:29
18365 03:35:30
18386 03:35:30
18345 03:35:31
18375 03:35:32
18355 03:35:33
18412 03:35:33
18392 03:35:34
18422 03:35:35
18402 03:35:35
18452 03:35:36
18462 03:35:37
18432 03:35:38
18442 03:35:38
18472 03:3

In [ ]:
from PIL import Image
# generate labels for validation
input_dir = "test/annotations"
output_dir = "test/labels/"
image_dir = "test/images/"
files = glob.glob(os.path.join(input_dir, '*.xml'))
for fil in files:
    basename = os.path.basename(fil)
    filename = os.path.splitext(basename)[0]
    if not os.path.exists(os.path.join(image_dir, f"{filename}.jpg")):
        print(f"{filename} image does not exist!")
        continue
    
    im = Image.open(os.path.join(image_dir, f"{filename}.jpg"))
    height, width = im.height, im.width
    
    result = []
    # parse the content of the xml file
    tree = ET.parse(fil)
    root = tree.getroot()


    for obj in root.findall('object'):
        label = obj.find("name").text
        # skip classes which are not seen in train data
        if label not in classes:
            continue
        index = classes.index(label)
        pil_bbox = [int(x.text) for x in obj.find("bndbox")]
        yolo_bbox = xml_to_yolo_bbox(pil_bbox, width, height)
        # convert data to string
        bbox_string = " ".join([str(x) for x in yolo_bbox])
        result.append(f"{index} {bbox_string}")

    if result:
        # generate a YOLO format text file for each xml file
        with open(os.path.join(output_dir, f"{filename}.txt"), "w", encoding="utf-8") as f:
            f.write("\n".join(result))


In [ ]:
# generate yaml file of the dataset
with open("data.yaml", "w", encoding="utf-8") as f:
  f.write("train: ANPR-2/train/images\n")
  f.write("val: ANPR-2/validation/images\n")
  f.write("test: ANPR-2/test/images\n\n")
  f.write("nc: "+ str(len(classes))+"\n")
  f.write("names: "+str(classes)+"\n")

In [ ]:
# we have the yolov7 model from first part, we just need to retrain it using the new dataset
# 10 epochs is enough to retrain the model
%cd /content/gdrive/MyDrive/yolov7
!python train.py --batch 16 --cfg cfg/training/yolov7.yaml --epochs 80 --data /content/gdrive/MyDrive/yolov7/ANPR-2/data.yaml --weights 'yolov7.pt' --device 0 


/content/gdrive/MyDrive/yolov7
YOLOR 🚀 v0.1-104-g941b94c torch 1.13.1+cu116 CUDA:0 (Tesla T4, 15101.8125MB)

Namespace(adam=False, artifact_alias='latest', batch_size=16, bbox_interval=-1, bucket='', cache_images=False, cfg='cfg/training/yolov7.yaml', data='/content/gdrive/MyDrive/yolov7/ANPR-2/data.yaml', device='0', entity=None, epochs=80, evolve=False, exist_ok=False, freeze=[0], global_rank=-1, hyp='data/hyp.scratch.p5.yaml', image_weights=False, img_size=[640, 640], label_smoothing=0.0, linear_lr=False, local_rank=-1, multi_scale=False, name='exp', noautoanchor=False, nosave=False, notest=False, project='runs/train', quad=False, rect=False, resume=False, save_dir='runs/train/exp6', save_period=-1, single_cls=False, sync_bn=False, total_batch_size=16, upload_dataset=False, weights='yolov7.pt', workers=8, world_size=1)
tensorboard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/
2023-03-02 08:01:12.900340: I tensorflow/core/platform/cpu_feature_guard.cc:

In [3]:
import torch
torch.cuda.empty_cache()
%cd /content/gdrive/MyDrive/yolov7
!python detect.py --weights /content/gdrive/MyDrive/yolov7/runs/train/exp6/weights/best.pt --conf 0.1 --source /content/gdrive/MyDrive/yolov7/ANPR-2/test/images --pil

Streaming output truncated to the last 5000 lines.
 The image with the result is saved in: runs/detect/exp7/12309.jpg
1 9, 1 5, 1 7, 1 3, 1 8, 1 6, 1 1, 1 و, Done. (12.2ms) Inference, (0.9ms) NMS
 The image with the result is saved in: runs/detect/exp7/12310.jpg
1 9, 1 8, 3 6s, 2 1s, 1 و, Done. (11.4ms) Inference, (0.9ms) NMS
 The image with the result is saved in: runs/detect/exp7/12319.jpg
1 9, 1 2, 1 5, 1 3, 1 6, 2 1s, 1 و, Done. (20.6ms) Inference, (2.1ms) NMS
 The image with the result is saved in: runs/detect/exp7/12320.jpg
2 9s, 1 5, 1 7, 1 3, 1 8, 1 1, 1 و, Done. (10.8ms) Inference, (1.0ms) NMS
 The image with the result is saved in: runs/detect/exp7/12329.jpg
1 9, 1 2, 2 5s, 1 4, 1 6, 1 1, 1 و, Done. (11.1ms) Inference, (0.9ms) NMS
 The image with the result is saved in: runs/detect/exp7/12330.jpg
1 9, 1 2, 1 5, 2 3s, 1 1, 1 0, 1 و, Done. (10.6ms) Inference, (0.9ms) NMS
 The image with the result is saved in: runs/detect/exp7/12339.jpg
2 9s, 1 8, 3 6s, 1 1, 1 و, Done. (10.4ms)

the model detects the charachters in the number plates of test data with nearly 100% precision and we can now use it to complete our solution. 

# second solution
train an image classification model to classify cropped characters in a number plate. 

load dataset of chars downloaded from https://github.com/alitourani/Iranis-dataset

In [ ]:
%cd /content/gdrive/MyDrive/persian_ocr
!mkdir dataset
!unzip IranisDatasetFiles.zip -d dataset

In [ ]:
class_names = subdirs = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'D', 'GH', 'H', 'J', 'L', 'M', 'N', 'P', 'PuV', 'PwD', 'Sad', 'Sin', 'T', 'Taxi', 'V', 'Y' ]
labels_nums = [i for i in range(0,len(class_names))]